In [2]:
from knapsackgym import KnapsackEnv
from AbstractKnapsackPolicy import AbstractKnapsackPolicy
from StateAggregator import StateAggregator
from typing import List, Callable, Optional, Union, Tuple, Dict, Any
from DP_Knapsack import solve_knapsack_dp, solve_KP_instances_with_DP
import numpy as np

## Main KnapsackDRLSolver

In [3]:
class KnapsackDRLSolver:
    """DRL-based Knapsack Solver using A2C algorithm"""
    
    def __init__(self, env, KPsolver:AbstractKnapsackPolicy, use_state_aggregation=False, verbose=True):
        """
        Initialize the DRL solver.
        
        Args:
            env: Gym environment for the knapsack problem
            N (int): Maximum number of items in a problem instance
            use_state_aggregation (bool): Whether to use state aggregation
            gamma (float): Discount factor
            lr_policy (float): Learning rate for policy network
            lr_value (float): Learning rate for value network
        """
        self.env:KnapsackEnv = env
        self.N = KPsolver.N
        self.use_state_aggregation = use_state_aggregation
        
        
        # Initialize state aggregator if needed
        self.state_aggregator = StateAggregator() if use_state_aggregation else None
        self.KPsolver = KPsolver
        self.verbose = verbose
        
    def process_state(self, state):
        """
        Process state with optional aggregation.
        
        Args:
            state (numpy.ndarray): Original state
            
        Returns:
            numpy.ndarray: Processed state
        """
        if self.use_state_aggregation and self.state_aggregator is not None:
            return self.state_aggregator.aggregate(state)
        return state
    
    def train(self, problem_instances, t_max=None):
        """
        Train the DRL solver on multiple problem instances with progress tracking.
        
        Args:
            problem_instances (List[Dict]): List of problem instances
            t_max (int): Maximum number of training steps
            
        Returns:
            List[float]: Values of solutions for each problem instance
        """
        assert len(problem_instances) is not None or len(problem_instances) > 0

        if t_max is None:
            t_max = 3 * self.N * 10000  # As specified in the pseudocode
        
        val:List[float] = [0] * len(problem_instances)  # Initialize solution values
        
        print(f"Training on {len(problem_instances)} KP Instances, with N={self.N}, t_max={t_max}")
        for t in range(t_max):

            # Select a problem instance (line 6 in pseudocode)
            # P_idx = np.random.randint(0, len(problem_instances))
            P_idx = t % len(problem_instances)
            P = problem_instances[P_idx]
            
            assert len(P['values'] )<= self.N, f"Problem Instance has too many items. KnapsackEnv is configuered to accept no more than <= {self.N}."

            # Change the environment to use this problem instance
            self.env.change_problem_instance(P)
            
            # Reset environment and get initial state
            state = self.env.reset()
            
            # Initialize for this episode
            done = False
            ow = 0  # Total weight of selected items
            ov = 0  # Total value of selected items
            
            # Create a copy of the problem instance P for modification
            n_P_prime = len(P['values'])
            W_P_prime = P['capacity']
            
            # Store states, actions, rewards for batch update
            states:List[np.ndarray] = []
            actions:List[int] = []
            rewards:List[float] = []
            next_states:List[np.ndarray] = []
            dones:List[bool] = []
            
            # Track episode rewards
            episode_rewards:List[float] = []
            
            # Solve the knapsack problem for this instance
            while ow < W_P_prime and n_P_prime > 0 and not done:
                # Process state if needed
                processed_state = self.process_state(state)
                states.append(processed_state)
                
                # Get available actions (indices of remaining items)
                available_actions = list(range(len(self.env.items)))
                
                # Get action according to policy (line 12 in pseudocode)
                action = self.KPsolver.get_action(processed_state, available_actions)
                actions.append(action)
                
                # Take action and observe reward and next state
                next_state, reward, done, info = self.env.step(action)
                rewards.append(reward)
                episode_rewards.append(reward)
                next_states.append(next_state)
                dones.append(done)
                
                # Check if item fits in knapsack (line 13 in pseudocode)
                if action < n_P_prime and info['current_weight'] - ow <= W_P_prime:
                    # Update totals (line 14 in pseudocode)
                    ow = info['current_weight']
                    ov = info['current_value']
                    W_P_prime = P['capacity'] - ow
                
                # Update P_prime (line 16 in pseudocode)
                n_P_prime -= 1
                
                # Update state
                state = next_state
            
            # Update parameters using collected trajectories
            self.KPsolver.update_parameters(states, actions, rewards, next_states, dones)
            
            if self.verbose and t % 1000 == 0:
                print(f"Iteration [{t}/{t_max}], Training KP Instance {P_idx}, Reward: {sum(rewards) / len(rewards)}")


            # Update best value if needed (lines 20-22 in pseudocode)
            if ov > val[P_idx]:
                val[P_idx] = ov
            
        return val
    
    
    def solve(self, problem_instance):
        """
        Solve a single knapsack problem instance using the trained policy.
        
        Args:
            problem_instance (Dict): A problem instance
            
        Returns:
            Tuple[float, List[int]]: Total value and list of selected item indices
        """
        # Set environment to use this problem instance
        self.env.change_problem_instance(problem_instance)
        
        # Reset environment and get initial state
        state = self.env.reset()
        
        done = False
        total_value = 0
        total_weight = 0
        selected_items = []
        
        while not done:
            # Process state if needed
            processed_state = self.process_state(state)
            
            # Get available actions (indices of remaining items)
            available_actions = list(range(len(self.env.items)))
            
            if not available_actions:
                break
                
            # Get action according to policy
            action = self.KPsolver.get_action(processed_state, available_actions)
            
            # Get original item index (before any removal)
            original_item_idx = self.env.items[action][2]
            
            # Take action and observe reward and next state
            next_state, reward, done, info = self.env.step(action)
            
            # If item was added (positive reward means item fit)
            if reward > 0:
                selected_items.append(original_item_idx)
            
            # Update value and weight
            total_value = info['current_value']
            total_weight = info['current_weight']
            
            # Update state
            state = next_state
        
        return total_value, selected_items

# Training Loop

In [4]:
from Greedy_Knapsack import greedy_knapsack, solve_problem_instances_greedy


def train_knapsack_solver(env, problem_instances:List[Dict[str, Any]], KPsolver, use_state_aggregation=False, t_max=None, verbose=True):
    """
    Train a DRL-based knapsack solver on multiple problem instances.
    
    Args:
        env: Gym environment for knapsack problem
        problem_instances (List[Dict]): List of problem instances
        N (int): Maximum number of items in a problem instance
        use_state_aggregation (bool): Whether to use state aggregation
        gamma (float): Discount factor
        lr_policy (float): Learning rate for policy network
        lr_value (float): Learning rate for value network
        t_max (int): Maximum number of training steps
        
    Returns:
        Tuple[KnapsackDRLSolver, List[float]]: Trained solver and solution values
    """
    # Initialize solver
    solver = KnapsackDRLSolver(
        env=env,
        KPsolver=KPsolver,
        use_state_aggregation=use_state_aggregation,
        verbose=verbose
    )
    
    # Train solver
    solution_values = solver.train(problem_instances, t_max)

    return solver, solution_values


def evaluate_knapsack_solver(solver:KnapsackDRLSolver, test_instances:list[dict]):
    """
    Evaluate the trained solver on test instances.
    
    Args:
        solver (KnapsackDRLSolver): Trained solver
        test_instances (List[Dict]): List of test problem instances
        
    Returns:
        List[Dict]: Evaluation results for each test instance
    """
    results = []
    
    for i, instance in enumerate(test_instances):
        # Solve instance
        value, selected_items = solver.solve(instance)
        
        # Calculate actual total weight
        total_weight = sum(instance['weights'][idx] for idx in selected_items)
        
        # Check if solution respects capacity constraint
        is_valid = total_weight <= instance['capacity']

        optimal_value, optimal_items = solve_knapsack_dp(instance)
        
        # Store results
        results.append({
            'instance_idx': i,
            'total_value': value,
            'total_weight': total_weight,
            'capacity': instance['capacity'],
            'selected_items': selected_items,
            'is_valid': is_valid,
            "optimal_sol": optimal_value,
            "optimal_items": optimal_items,
            "optimality_ratio": value /  optimal_value 
        })
    
    return results

def run_KPSolver(env:KnapsackEnv, KPSolver:KnapsackDRLSolver, 
                 training_problem_instances: List[Dict[str, Any]], t_max:int=None) -> List[float]:
    
    print(f"Running Model {KPSolver.__class__}")
    
    # Train solver
    solver, solution_values = train_knapsack_solver(
        env=env,
        problem_instances=training_problem_instances,
        KPsolver=KPSolver,
        use_state_aggregation=False,
        t_max=t_max,
        verbose=True
    )
    
    return solution_values


def print_results(results:list):
    print("Evaluation results:")
    for res in results: 
        print(res)


## Testing stuff

In [ ]:
from KnapsackPPO import KnapsackPPOSolver
from A2C import KnapsackA2C
from KnapsackQLearning import KnapsackDQN
from util.instance_gen import KnapsackInstanceGenerator
from util.metrics import evaluate_knapsack_performance

# TODO integrate the instance generator in this code

# Example usage
if __name__ == "__main__":
    

    N = 50
    gamma = 0.99
    # t_max = 10000
    t_max = None

    env:KnapsackEnv = KnapsackEnv(problem_instance=None, N=N)
    gen = KnapsackInstanceGenerator(seed=42)

    problem_instances = gen.generate('RI', M=3, N=N, R=100)
    print(problem_instances)

    KPSolver_A2C = KnapsackA2C(N=N, gamma=gamma, lr_policy=0.001, lr_value=0.001, verbose=False)
    KPSolver_PPO = KnapsackPPOSolver(N=N, gamma=gamma, policy_lr=0.001, value_lr=0.001, verbose=False)
    KPSolver_DQN = KnapsackDQN(N=N, gamma=gamma, lr=0.001, verbose=False)

    DP_sol_items, DP_value, DP_weight = solve_KP_instances_with_DP(problem_instances)
    Greedy_value_total, Greedy_selected, Greedy_weight_total = solve_problem_instances_greedy(problem_instances)
    
    
    A2C_Results = run_KPSolver(env=env, KPSolver=KPSolver_A2C, training_problem_instances=problem_instances, t_max=t_max)
    PPO_Results = run_KPSolver(env=env, KPSolver=KPSolver_PPO, training_problem_instances=problem_instances, t_max=t_max)
    DQN_Results = run_KPSolver(env=env, KPSolver=KPSolver_DQN, training_problem_instances=problem_instances, t_max=t_max)

    # print(A2C_Results, DP_value,  Greedy_value_total)
    A2C_metric_results = evaluate_knapsack_performance(A2C_Results, DP_value, Greedy_value_total)
    PPO_metric_results = evaluate_knapsack_performance(PPO_Results, DP_value, Greedy_value_total)
    DQN_metric_results = evaluate_knapsack_performance(DQN_Results, DP_value, Greedy_value_total)

    print("Optimal solution values:", DP_value)
    print("Greedy solution values:", Greedy_value_total)
    print("Trained solution values:", A2C_Results)
    print("Trained solution values:", PPO_Results)
    print("Trained solution values:", DQN_Results)

    print(A2C_metric_results)
    print(PPO_metric_results)
    print(DQN_metric_results)




[{'values': [9, 70, 21, 10, 53], 'weights': [78, 66, 44, 44, 86], 'capacity': 293}, {'values': [76, 20, 37, 47, 50, 5, 55, 16, 75, 69, 93, 75, 37, 97, 42, 33, 91, 38, 8, 47, 80, 19, 47, 13, 69, 48, 34, 23, 57, 67, 95, 44, 17, 84, 63, 71, 10], 'weights': [77, 72, 79, 52, 13, 84, 46, 51, 38, 19, 93, 79, 65, 41, 83, 55, 45, 46, 23, 10, 56, 89, 7, 86, 83, 28, 64, 17, 76, 71, 36, 7, 98, 45, 90, 68, 78], 'capacity': 100}, {'values': [58, 64, 57, 56, 10, 56, 80, 31, 61, 4, 35, 44, 99, 22, 28, 41, 100, 86, 4, 24, 83, 6, 86, 29, 92, 30, 44, 67, 13, 56, 51, 79, 100, 67, 41, 41, 42, 82, 33], 'weights': [84, 44, 81, 85, 39, 90, 29, 24, 69, 64, 14, 84, 20, 81, 1, 80, 79, 79, 67, 48, 71, 28, 79, 56, 46, 51, 57, 4, 14, 25, 12, 44, 67, 66, 48, 86, 57, 8, 77], 'capacity': 58}]
Running Model <class 'A2C.KnapsackA2C'>
Training on 3 KP Instances, with N=50, t_max=1500000
Iteration [0/1500000], Training KP Instance 0, Reward: 0.0012485728415564117
Iteration [1000/1500000], Training KP Instance 1, Reward: 0